In [1]:
import json
import scipy
import copy
import operator
import math
from scipy import stats

In [2]:
typeDict = {
    u'日式料理':[],
    u'牛排館/鐵板燒':[],
    u'主題/景觀餐廳':[],
    u'台式/中式/小吃':[],
    u'合菜':[],
    u'羊肉爐/牛肉湯':[],
    u'自助餐':[],
    u'串燒/碳烤/居酒屋/燒烤':[],
    u'咖啡/輕食/下午茶':[],
    u'東南亞料理':[],
    u'美式餐廳/早午餐':[],
    u'海產/熱炒餐廳':[],
    u'烤雞/烤鴨':[],
    u'素食/蔬食':[],
    u'酒吧/餐酒館':[],
    u'甜點/冰品/飲料/烘培':[],
    u'港式餐廳':[],
    u'義式/歐風/異國':[],
    u'鍋類':[],
    u'韓式料理':[],
    u'其他':[]
}

In [3]:
with open('./Workspace/result/UserWithRegion.json', 'r') as f:
    user = json.load(f)
with open('./Workspace/result/BlogWithType.json', 'r') as f:
    blog = json.load(f)

In [4]:
for b in blog:
    score = blog[b]['score']
    sentiment_dict = user[blog[b]['author_id']].get('sentiment_score')
    if not sentiment_dict:
        user[blog[b]['author_id']]['sentiment_score'] = copy.deepcopy(typeDict)
        user[blog[b]['author_id']]['sentiment_score'][blog[b]['restaurant_type']].append(score)
    else:
        user[blog[b]['author_id']]['sentiment_score'][blog[b]['restaurant_type']].append(score)

In [24]:
print user['55f7699c2756dd40def8c2a4']['sentiment_score'][u'酒吧/餐酒館']

[0]


In [11]:
zero_dict = copy.deepcopy(typeDict)
for ele in zero_dict:
    zero_dict[ele] = 0
for u in user:
    user[u]['blog_cnt_by_type'] = {}
    if not user[u].get('sentiment_score'):
        user[u]['blog_cnt_by_type'] = copy.deepcopy(zero_dict)
    else:
        sc = user[u]['sentiment_score']
        for item in sc:
            user[u]['blog_cnt_by_type'][item] = len(sc[item])

In [12]:
print user['5732d5d1699b6e18f5a7c9ae']['blog_cnt_by_type']

{u'\u70e4\u96de/\u70e4\u9d28': 0, u'\u4e32\u71d2/\u78b3\u70e4/\u5c45\u9152\u5c4b/\u71d2\u70e4': 0, u'\u6d77\u7522/\u71b1\u7092\u9910\u5ef3': 0, u'\u7d20\u98df/\u852c\u98df': 0, u'\u7f8e\u5f0f\u9910\u5ef3/\u65e9\u5348\u9910': 0, u'\u5408\u83dc': 0, u'\u6e2f\u5f0f\u9910\u5ef3': 0, u'\u65e5\u5f0f\u6599\u7406': 0, u'\u6771\u5357\u4e9e\u6599\u7406': 0, u'\u7f8a\u8089\u7210/\u725b\u8089\u6e6f': 0, u'\u725b\u6392\u9928/\u9435\u677f\u71d2': 1, u'\u97d3\u5f0f\u6599\u7406': 0, u'\u9152\u5427/\u9910\u9152\u9928': 0, u'\u7fa9\u5f0f/\u6b50\u98a8/\u7570\u570b': 0, u'\u5176\u4ed6': 0, u'\u751c\u9ede/\u51b0\u54c1/\u98f2\u6599/\u70d8\u57f9': 0, u'\u4e3b\u984c/\u666f\u89c0\u9910\u5ef3': 0, u'\u53f0\u5f0f/\u4e2d\u5f0f/\u5c0f\u5403': 0, u'\u81ea\u52a9\u9910': 0, u'\u934b\u985e': 1, u'\u5496\u5561/\u8f15\u98df/\u4e0b\u5348\u8336': 0}


In [30]:
for u in user:
    sc = user[u].get('sentiment_score')
    if sc:
        item = []
        for ele in sc:
            item.extend(map(lambda x: (ele, x), sc[ele]))
            if sc[ele]:
                sc[ele] = scipy.mean(sc[ele])
            else:
                sc[ele] = None
        scores = map(operator.itemgetter(1), item)
        types = map(operator.itemgetter(0), item)
        zc = [z if not math.isnan(z) else 0 for z in stats.zscore(scores)]
        user[u]['within_zscore'] = {}
        count_dict = {}
        for e in zip(types, zc):
            if e[0] not in user[u]['within_zscore']:
                user[u]['within_zscore'][e[0]] = e[1]
                count_dict[e[0]] = [e[1]]
            else:
                n = float(len(count_dict[e[0]]))
                user[u]['within_zscore'][e[0]] = user[u]['within_zscore'][e[0]] * n / (n+1) + e[1] / (n+1)
                count_dict[e[0]].append([e[1]])
        for key in typeDict:
            if key not in user[u]['within_zscore']:
                user[u]['within_zscore'][key] = -999

/Users/fan/anaconda/lib/python2.7/site-packages/scipy/stats/stats.py:2480: RuntimeWarning: invalid value encountered in true_divide
  return (a - mns) / sstd


In [31]:
print user['5732d5d1699b6e18f5a7c9ae']['within_zscore']

{u'\u70e4\u96de/\u70e4\u9d28': -999, u'\u7f8a\u8089\u7210/\u725b\u8089\u6e6f': -999, u'\u4e3b\u984c/\u666f\u89c0\u9910\u5ef3': -999, u'\u6d77\u7522/\u71b1\u7092\u9910\u5ef3': -999, u'\u5176\u4ed6': -999, u'\u5408\u83dc': -999, u'\u5496\u5561/\u8f15\u98df/\u4e0b\u5348\u8336': -999, u'\u65e5\u5f0f\u6599\u7406': -999, u'\u6771\u5357\u4e9e\u6599\u7406': -999, u'\u4e32\u71d2/\u78b3\u70e4/\u5c45\u9152\u5c4b/\u71d2\u70e4': -999, u'\u725b\u6392\u9928/\u9435\u677f\u71d2': 0, u'\u97d3\u5f0f\u6599\u7406': -999, u'\u9152\u5427/\u9910\u9152\u9928': -999, u'\u7fa9\u5f0f/\u6b50\u98a8/\u7570\u570b': -999, u'\u7f8e\u5f0f\u9910\u5ef3/\u65e9\u5348\u9910': -999, u'\u751c\u9ede/\u51b0\u54c1/\u98f2\u6599/\u70d8\u57f9': -999, u'\u7d20\u98df/\u852c\u98df': -999, u'\u53f0\u5f0f/\u4e2d\u5f0f/\u5c0f\u5403': -999, u'\u81ea\u52a9\u9910': -999, u'\u934b\u985e': 0, u'\u6e2f\u5f0f\u9910\u5ef3': -999}


In [209]:
# for u in user:
#     sc = user[u].get('sentiment_score')
#     if sc:
#         for ele in sc:
#             if sc[ele]:
#                 sc[ele] = scipy.mean(sc[ele])
#             else:
#                 sc[ele] = None

In [32]:
print user['5732d5d1699b6e18f5a7c9ae']['sentiment_score']

{u'\u70e4\u96de/\u70e4\u9d28': None, u'\u7f8a\u8089\u7210/\u725b\u8089\u6e6f': None, u'\u4e3b\u984c/\u666f\u89c0\u9910\u5ef3': None, u'\u6d77\u7522/\u71b1\u7092\u9910\u5ef3': None, u'\u5176\u4ed6': None, u'\u5408\u83dc': None, u'\u5496\u5561/\u8f15\u98df/\u4e0b\u5348\u8336': None, u'\u65e5\u5f0f\u6599\u7406': None, u'\u6771\u5357\u4e9e\u6599\u7406': None, u'\u4e32\u71d2/\u78b3\u70e4/\u5c45\u9152\u5c4b/\u71d2\u70e4': None, u'\u725b\u6392\u9928/\u9435\u677f\u71d2': 14.0, u'\u97d3\u5f0f\u6599\u7406': None, u'\u9152\u5427/\u9910\u9152\u9928': None, u'\u7fa9\u5f0f/\u6b50\u98a8/\u7570\u570b': None, u'\u7f8e\u5f0f\u9910\u5ef3/\u65e9\u5348\u9910': None, u'\u751c\u9ede/\u51b0\u54c1/\u98f2\u6599/\u70d8\u57f9': None, u'\u7d20\u98df/\u852c\u98df': None, u'\u53f0\u5f0f/\u4e2d\u5f0f/\u5c0f\u5403': None, u'\u81ea\u52a9\u9910': None, u'\u934b\u985e': 14.0, u'\u6e2f\u5f0f\u9910\u5ef3': None}


In [211]:
# for u in user:
#     typeList = []
#     tempList = []
#     sc = user[u].get('sentiment_score')
#     if sc:
#         for ele in sc:
#             if sc[ele]:
#                 typeList.append(ele)
#                 tempList.append(sc[ele])
#         type_name = typeList
#         type_zscore = [z if not math.isnan(z) else 0 for z in stats.zscore(tempList)]
#         user[u]['within_zscore'] = dict(zip(type_name, type_zscore))
#         for key in typeDict:
#             if key not in user[u]['within_zscore']:
#                 user[u]['within_zscore'][key] = -999 

In [33]:
tempDict = copy.deepcopy(typeDict)
for u in user:
    sc = user[u].get('sentiment_score')
    if sc:
        for ele in sc:
            if sc.get(ele) or sc.get(ele) == 0:
                tempDict[ele].append((u, sc[ele]))

In [34]:
score_dict = copy.deepcopy(typeDict)
for ele in score_dict:
    score_dict[ele] = -999
for ele in tempDict:
    uid = [score[0] for score in tempDict[ele]]
    zscore = stats.zscore([score[1] for score in tempDict[ele]])
    zscore_pkg = zip(uid, zscore)
    for tp in zscore_pkg:    
        bz = user[tp[0]].get('between_zscore')
        if not bz:
            user[tp[0]]['between_zscore'] = copy.deepcopy(score_dict)
            user[tp[0]]['between_zscore'][ele] = tp[1]
        else:
            user[tp[0]]['between_zscore'][ele] = tp[1]

In [35]:
testid = '55f7699c2756dd40def8c2a4'

In [36]:
print user[testid]['display_name'], 'between_zscore'
print '='*50
for ele in user[testid]['between_zscore'].items():
    print ele[0], ele[1]

跳躍的宅男 between_zscore
烤雞/烤鴨 -1.13707132945
串燒/碳烤/居酒屋/燒烤 -1.05320372477
海產/熱炒餐廳 -0.196909122815
素食/蔬食 -999
美式餐廳/早午餐 -0.458715249468
合菜 -0.283023973997
港式餐廳 -1.10399669554
日式料理 -0.590278706889
東南亞料理 -0.65776950216
羊肉爐/牛肉湯 1.02891110999
牛排館/鐵板燒 -0.675907217581
韓式料理 -999
酒吧/餐酒館 -1.60920613663
義式/歐風/異國 -0.727258895915
其他 -0.833993667778
甜點/冰品/飲料/烘培 -1.2274669712
主題/景觀餐廳 -0.495621986638
台式/中式/小吃 -0.562177999834
自助餐 -1.37358638596
鍋類 -0.485545390459
咖啡/輕食/下午茶 -0.972781852969


In [37]:
print user[testid]['display_name'], 'within_zscore'
print '='*50
for ele in user[testid]['within_zscore'].items():
    print ele[0], ele[1]

跳躍的宅男 within_zscore
烤雞/烤鴨 -0.521861146897
串燒/碳烤/居酒屋/燒烤 0.0664949345914
海產/熱炒餐廳 0.645437318776
素食/蔬食 -999
美式餐廳/早午餐 0.301475301906
合菜 1.14436327588
港式餐廳 -0.465378963074
日式料理 0.246296553094
東南亞料理 0.297130518535
羊肉爐/牛肉湯 2.15162888738
牛排館/鐵板燒 0.494818161915
韓式料理 -999
酒吧/餐酒館 -1.36909390424
義式/歐風/異國 0.417155159158
其他 -0.917236433656
甜點/冰品/飲料/烘培 -0.648115440148
主題/景觀餐廳 0.0853223291991
台式/中式/小吃 -0.188889563231
自助餐 -0.804272066011
鍋類 0.640730470124
咖啡/輕食/下午茶 -0.230977900209


In [38]:
print user[testid]['display_name'], 'blog_cnt'
print '='*50
for ele in user[testid]['blog_cnt_by_type'].items():
    print ele[0], ele[1]

跳躍的宅男 blog_cnt
烤雞/烤鴨 1
串燒/碳烤/居酒屋/燒烤 6
海產/熱炒餐廳 9
素食/蔬食 0
美式餐廳/早午餐 13
合菜 6
港式餐廳 1
日式料理 5
東南亞料理 5
羊肉爐/牛肉湯 3
牛排館/鐵板燒 1
韓式料理 0
酒吧/餐酒館 1
義式/歐風/異國 4
其他 2
甜點/冰品/飲料/烘培 17
主題/景觀餐廳 2
台式/中式/小吃 62
自助餐 1
鍋類 6
咖啡/輕食/下午茶 20


In [39]:
score_dict = copy.deepcopy(typeDict)
for ele in score_dict:
    score_dict[ele] = -999
for ele in user:
    if not user[ele].get('between_zscore'):
        user[ele]['between_zscore'] = copy.deepcopy(score_dict)
    if not user[ele].get('within_zscore'):
        user[ele]['within_zscore'] = copy.deepcopy(score_dict)

In [40]:
with open('./Workspace/result/UserWithZscoreV2.json', 'w') as f:
    json.dump(user,f)